In [1]:
!pip install order_of_magnitude
!pip install netCDF4

  Preparing metadata (setup.py) ... done
  Created wheel for order_of_magnitude: filename=order_of_magnitude-2.3.1-py3-none-any.whl size=4579 sha256=19fb522e65181c4bf3bb4f3eabb8bc65c729f2e10b37efb8589a188fe868d2f3
  Stored in directory: /root/.cache/pip/wheels/85/53/9b/310924c3bd35596e70e6d3597f347986a2f95cfb6393e374ec
Successfully built order_of_magnitude
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/ERA5_Dataset")

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import netCDF4 as nc

## Target Dataframe

In [4]:
df_target = pd.read_csv('./csv/target_NoOutliers_complete.csv')

lats = df_target['LAT'].values
lons = df_target['LON'].values

df_target

,DATE,ID,LAT,LON,S.IndAll
0,1980-01-01,NaN,-18.50,80.50,0.00
1,1980-01-02,NaN,-30.00,31.25,0.00
2,1980-01-03,NaN,-4.00,86.50,0.00
3,1980-01-04,NaN,-29.50,32.75,0.00
4,1980-01-05,NaN,-4.75,89.25,0.00
...,...,...,...,...,...
15422,2022-03-23,392.0,-13.75,76.75,0.50
15423,2022-03-24,392.0,-14.25,74.25,1.00
15424,2022-03-25,392.0,-13.50,73.25,1.00
15425,2022-03-26,392.0,-15.00,74.00,1.00


In [5]:
dates = df_target['DATE'].values
N_dates = dates.shape[0]
print(N_dates)

15427


## Wind Speed

In [6]:
ds_wind = nc.Dataset('../ERA5_Dataset/nc_original_datasets/wind_speed_10m.nc')

In [7]:
ds_wind['u10']

<class 'netCDF4._netCDF4.Variable'>
int16 u10(time, expver, latitude, longitude)
    scale_factor: 0.0010146746628815588
    add_offset: -2.1994132821556596
    _FillValue: -32767
    missing_value: -32767
    units: m s**-1
    long_name: 10 metre U wind component
unlimited dimensions: 
current shape = (15990, 2, 121, 241)
filling on

In [8]:
wind_u = ds_wind['u10'][:N_dates, 0, :, :]
wind_v = ds_wind['v10'][:N_dates, 0, :, :]

In [9]:
print(wind_u.shape, wind_v.shape)

(15427, 121, 241) (15427, 121, 241)


In [10]:
u_wind_zones = []

u_wind_zones.append(wind_u[:N_dates, 0:30, 0:60])
u_wind_zones.append(wind_u[:N_dates, 0:30, 60:120])
u_wind_zones.append(wind_u[:N_dates, 0:30, 120:180])
u_wind_zones.append(wind_u[:N_dates, 0:30, 180:])

u_wind_zones.append(wind_u[:N_dates, 30:60, 0:60])
u_wind_zones.append(wind_u[:N_dates, 30:60, 60:120])
u_wind_zones.append(wind_u[:N_dates, 30:60, 120:180])
u_wind_zones.append(wind_u[:N_dates, 30:60, 180:])

u_wind_zones.append(wind_u[:N_dates, 60:90, 0:60])
u_wind_zones.append(wind_u[:N_dates, 60:90, 60:120])
u_wind_zones.append(wind_u[:N_dates, 60:90, 120:180])
u_wind_zones.append(wind_u[:N_dates, 60:90, 180:])

u_wind_zones.append(wind_u[:N_dates, 90:, 0:60])
u_wind_zones.append(wind_u[:N_dates, 90:, 60:120])
u_wind_zones.append(wind_u[:N_dates, 90:, 120:180])
u_wind_zones.append(wind_u[:N_dates, 90:, 180:])

print(len(u_wind_zones))

v_wind_zones = []

v_wind_zones.append(wind_v[:N_dates, 0:30, 0:60])
v_wind_zones.append(wind_v[:N_dates, 0:30, 60:120])
v_wind_zones.append(wind_v[:N_dates, 0:30, 120:180])
v_wind_zones.append(wind_v[:N_dates, 0:30, 180:])

v_wind_zones.append(wind_v[:N_dates, 30:60, 0:60])
v_wind_zones.append(wind_v[:N_dates, 30:60, 60:120])
v_wind_zones.append(wind_v[:N_dates, 30:60, 120:180])
v_wind_zones.append(wind_v[:N_dates, 30:60, 180:])

v_wind_zones.append(wind_v[:N_dates, 60:90, 0:60])
v_wind_zones.append(wind_v[:N_dates, 60:90, 60:120])
v_wind_zones.append(wind_v[:N_dates, 60:90, 120:180])
v_wind_zones.append(wind_v[:N_dates, 60:90, 180:])

v_wind_zones.append(wind_v[:N_dates, 90:, 0:60])
v_wind_zones.append(wind_v[:N_dates, 90:, 60:120])
v_wind_zones.append(wind_v[:N_dates, 90:, 120:180])
v_wind_zones.append(wind_v[:N_dates, 90:, 180:])

print(len(v_wind_zones))

16
16


In [11]:
u_mean_zones = []
u_std_zones  = []
v_mean_zones = []
v_std_zones  = []

for zone in range(16):
  means = []
  stds = []
  for date_idx in range(N_dates):
    temp_mean = np.mean(u_wind_zones[zone][date_idx])
    temp_std = np.std(u_wind_zones[zone][date_idx])
    means.append(temp_mean)
    stds.append(temp_std)
  u_mean_zones.append(means)
  u_std_zones.append(stds)

for zone in range(16):
  means = []
  stds = []
  for date_idx in range(N_dates):
    temp_mean = np.mean(v_wind_zones[zone][date_idx])
    temp_std = np.std(v_wind_zones[zone][date_idx])
    means.append(temp_mean)
    stds.append(temp_std)
  v_mean_zones.append(means)
  v_std_zones.append(stds)

In [12]:
u_mean_std_zones = np.column_stack((u_mean_zones[0], u_std_zones[0],
                                    u_mean_zones[1], u_std_zones[1],
                                    u_mean_zones[2], u_std_zones[2],
                                    u_mean_zones[3], u_std_zones[3],
                                    u_mean_zones[4], u_std_zones[4],
                                    u_mean_zones[5], u_std_zones[5],
                                    u_mean_zones[6], u_std_zones[6],
                                    u_mean_zones[7], u_std_zones[7],
                                    u_mean_zones[8], u_std_zones[8],
                                    u_mean_zones[9], u_std_zones[9],
                                    u_mean_zones[10], u_std_zones[10],
                                    u_mean_zones[11], u_std_zones[11],
                                    u_mean_zones[12], u_std_zones[12],
                                    u_mean_zones[13], u_std_zones[13],
                                    u_mean_zones[14], u_std_zones[14],
                                    u_mean_zones[15], u_std_zones[15]))

v_mean_std_zones = np.column_stack((v_mean_zones[0], v_std_zones[0],
                                    v_mean_zones[1], v_std_zones[1],
                                    v_mean_zones[2], v_std_zones[2],
                                    v_mean_zones[3], v_std_zones[3],
                                    v_mean_zones[4], v_std_zones[4],
                                    v_mean_zones[5], v_std_zones[5],
                                    v_mean_zones[6], v_std_zones[6],
                                    v_mean_zones[7], v_std_zones[7],
                                    v_mean_zones[8], v_std_zones[8],
                                    v_mean_zones[9], v_std_zones[9],
                                    v_mean_zones[10], v_std_zones[10],
                                    v_mean_zones[11], v_std_zones[11],
                                    v_mean_zones[12], v_std_zones[12],
                                    v_mean_zones[13], v_std_zones[13],
                                    v_mean_zones[14], v_std_zones[14],
                                    v_mean_zones[15], v_std_zones[15]))

# Save Datset

In [13]:
target_att = ['DATE', 'S.IndAll']
mean_std_zones_att = ['Mean_Z1', 'Std_Z1',
                      'Mean_Z2', 'Std_Z2',
                      'Mean_Z3', 'Std_Z3',
                      'Mean_Z4', 'Std_Z4',
                      'Mean_Z5', 'Std_Z5',
                      'Mean_Z6', 'Std_Z6',
                      'Mean_Z7', 'Std_Z7',
                      'Mean_Z8', 'Std_Z8',
                      'Mean_Z9', 'Std_Z9',
                      'Mean_Z10', 'Std_Z10',
                      'Mean_Z11', 'Std_Z11',
                      'Mean_Z12', 'Std_Z12',
                      'Mean_Z13', 'Std_Z13',
                      'Mean_Z14', 'Std_Z14',
                      'Mean_Z15', 'Std_Z15',
                      'Mean_Z16', 'Std_Z16']

target = df_target[target_att].values
u_final_array = np.hstack((target, u_mean_std_zones))
df_complete_u = pd.DataFrame(u_final_array, columns=target_att+mean_std_zones_att)
df_complete_u[11:]

,DATE,S.IndAll,Mean_Z1,Std_Z1,Mean_Z2,Std_Z2,Mean_Z3,Std_Z3,Mean_Z4,Std_Z4,...,Mean_Z12,Std_Z12,Mean_Z13,Std_Z13,Mean_Z14,Std_Z14,Mean_Z15,Std_Z15,Mean_Z16,Std_Z16
11,1980-01-12,0.0,-1.696971,2.46277,0.745667,3.010941,1.898294,2.786089,4.504408,2.958747,...,-6.423232,1.371531,-2.800083,1.729003,-2.683587,3.527298,-9.329498,1.021441,-7.397588,1.542429
12,1980-01-13,0.0,-1.940168,2.71582,-1.288469,2.863892,0.577588,2.634022,4.526747,3.097227,...,-7.585702,1.046703,-2.089551,1.476581,-2.146021,4.029759,-9.630692,0.8378,-7.089679,1.26081
13,1980-01-14,0.0,-2.275212,2.227479,-1.102822,3.055544,2.219861,2.960348,4.915563,2.620962,...,-8.281836,0.944343,-3.253654,2.136057,-3.113518,2.243333,-8.878569,1.578549,-7.39011,1.476831
14,1980-01-15,0.0,-1.943522,2.322055,-0.678897,2.860336,2.297576,3.240568,5.261277,3.099225,...,-9.183595,0.968233,-4.059362,2.276793,-3.032676,2.059227,-6.917282,1.623635,-7.321972,2.166364
15,1980-01-16,0.0,-2.001346,2.219746,-0.454847,2.983891,2.0809,2.614041,4.837062,2.968727,...,-7.827386,0.999029,-6.721549,3.151608,-5.031947,2.014183,-3.009147,2.638866,-6.358754,2.017236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15422,2022-03-23,0.5,-0.903472,1.911197,-0.338967,2.283941,4.608987,1.346626,4.821469,2.499482,...,-10.157874,1.100313,-4.962925,3.177425,-5.725322,2.000138,-5.392983,1.541616,-8.326871,1.249284
15423,2022-03-24,1.0,-1.291725,1.672339,1.087489,2.717875,7.097342,1.51059,3.595105,2.371297,...,-8.861173,1.447017,-2.923171,3.161113,-5.95921,2.095521,-5.605412,1.032399,-6.599097,1.331947
15424,2022-03-25,1.0,-1.387864,1.27287,1.559053,2.461469,7.200207,1.730253,2.45039,2.054298,...,-8.460263,1.755847,1.926,2.290293,-3.161103,1.777174,-7.308817,1.139927,-6.909372,1.94619
15425,2022-03-26,1.0,-1.896455,1.712293,0.844224,2.769472,5.621906,1.471154,0.161213,2.331351,...,-8.194703,2.096412,-2.603624,2.187073,-0.129043,2.766286,-5.978816,1.941825,-8.742082,1.520525


In [14]:
target = df_target[target_att].values
v_final_array = np.hstack((target, v_mean_std_zones))
df_complete_v = pd.DataFrame(v_final_array, columns=target_att+mean_std_zones_att)
df_complete_v[11:]

,DATE,S.IndAll,Mean_Z1,Std_Z1,Mean_Z2,Std_Z2,Mean_Z3,Std_Z3,Mean_Z4,Std_Z4,...,Mean_Z12,Std_Z12,Mean_Z13,Std_Z13,Mean_Z14,Std_Z14,Mean_Z15,Std_Z15,Mean_Z16,Std_Z16
11,1980-01-12,0.0,-1.267676,2.280404,-5.507282,1.507224,-4.494953,1.264765,-1.048716,2.738511,...,2.888565,1.147426,4.163404,4.453504,-2.874276,7.093936,-1.603666,2.128822,4.616043,1.652546
12,1980-01-13,0.0,-0.633654,2.716036,-4.721637,1.14577,-2.02155,1.24475,0.276025,2.110439,...,3.240099,1.099109,1.753071,4.026045,-1.410596,5.748139,-2.019369,2.068879,3.783522,1.465097
13,1980-01-14,0.0,-0.802254,2.282864,-3.393209,0.828131,-2.621409,1.093307,-2.075183,2.229433,...,3.465081,0.962018,2.635149,2.602474,0.887271,4.516126,-3.020072,2.099575,3.049986,1.710988
14,1980-01-15,0.0,-1.421811,2.864186,-4.345302,1.039041,-3.48359,0.845879,-2.605149,2.027639,...,1.940918,0.826392,3.164221,2.432545,2.370998,3.94555,-4.159219,1.598572,0.823769,2.119534
15,1980-01-16,0.0,-0.479325,2.218676,-3.003993,0.832316,-2.768578,1.335009,-1.62895,2.339194,...,0.270266,1.424782,1.875077,3.124364,7.427509,3.726662,-5.701421,3.217623,-1.163542,2.505501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15422,2022-03-23,0.5,2.131373,1.661002,1.622151,2.228892,-2.234183,2.160619,-2.305886,1.820508,...,3.157291,1.25113,1.987415,2.431633,1.458633,1.966493,0.026792,1.700225,0.89883,1.682846
15423,2022-03-24,1.0,2.224319,2.089805,1.84939,2.224406,-2.312389,1.983654,-2.316264,2.649745,...,2.877192,2.259021,-1.848554,2.304234,0.591985,2.268476,1.457583,1.177019,1.485987,1.413358
15424,2022-03-25,1.0,1.576384,1.67853,1.938625,1.723324,-1.729534,2.362857,-3.705136,3.045288,...,1.367778,1.860073,-0.723876,3.665415,-3.741538,2.494533,0.323047,1.886015,3.069456,1.360739
15425,2022-03-26,1.0,0.491059,1.738263,-0.263019,2.103198,-2.7248,2.538858,-2.55717,2.487697,...,-1.149152,3.159007,2.97354,1.736519,-2.288397,2.987579,-0.857055,2.428804,0.966657,1.912848


# Rollback 10 days

In [15]:
def attribute_stepped(atts, max_steps):
  att_list = []
  for att in atts:
    for i in range(max_steps):
      att_list.append('{}-{}'.format(att,i))
  return att_list

In [16]:
def mean_std_atts_zones(zones_n):
  new_att_list = []
  for i in range(zones_n):
    new_att_list.append('Mean_Z{}'.format(i+1))
    new_att_list.append('Std_Z{}'.format(i+1))
  return new_att_list

print(mean_std_atts_zones(16))

['Mean_Z1', 'Std_Z1', 'Mean_Z2', 'Std_Z2', 'Mean_Z3', 'Std_Z3', 'Mean_Z4', 'Std_Z4', 'Mean_Z5', 'Std_Z5', 'Mean_Z6', 'Std_Z6', 'Mean_Z7', 'Std_Z7', 'Mean_Z8', 'Std_Z8', 'Mean_Z9', 'Std_Z9', 'Mean_Z10', 'Std_Z10', 'Mean_Z11', 'Std_Z11', 'Mean_Z12', 'Std_Z12', 'Mean_Z13', 'Std_Z13', 'Mean_Z14', 'Std_Z14', 'Mean_Z15', 'Std_Z15', 'Mean_Z16', 'Std_Z16']


In [17]:
max_steps = 11
labels_to_rewind = mean_std_atts_zones(16)
labels_stepped = attribute_stepped(labels_to_rewind, max_steps)
print(labels_stepped)

stepped_mean_zones = []
stepped_std_zones = []

for zone in range(1, 17):
  mean_press_10d = np.zeros((N_dates-max_steps, max_steps))
  std_press_10d = np.zeros((N_dates-max_steps, max_steps))

  mean_press = df_complete_u['Mean_Z{}'.format(zone)]
  std_press = df_complete_u['Std_Z{}'.format(zone)]

  for day in range(max_steps, N_dates):
    for steps in range(max_steps):
      mean_press_10d[day-max_steps, steps] = mean_press[day-steps]
      std_press_10d[day-max_steps, steps] = std_press[day-steps]
  print(mean_press_10d.shape)
  stepped_mean_zones.append(mean_press_10d)
  stepped_std_zones.append(std_press_10d)

final_array = np.hstack((target[max_steps:],
                         stepped_mean_zones[0], stepped_std_zones[0],
                         stepped_mean_zones[1], stepped_std_zones[1],
                         stepped_mean_zones[2], stepped_std_zones[2],
                         stepped_mean_zones[3], stepped_std_zones[3],
                         stepped_mean_zones[4], stepped_std_zones[4],
                         stepped_mean_zones[5], stepped_std_zones[5],
                         stepped_mean_zones[6], stepped_std_zones[6],
                         stepped_mean_zones[7], stepped_std_zones[7],
                         stepped_mean_zones[8], stepped_std_zones[8],
                         stepped_mean_zones[9], stepped_std_zones[9],
                         stepped_mean_zones[10], stepped_std_zones[10],
                         stepped_mean_zones[11], stepped_std_zones[11],
                         stepped_mean_zones[12], stepped_std_zones[12],
                         stepped_mean_zones[13], stepped_std_zones[13],
                         stepped_mean_zones[14], stepped_std_zones[14],
                         stepped_mean_zones[15], stepped_std_zones[15],
                         ))
df_stepped_u = pd.DataFrame(final_array , columns = target_att + labels_stepped)
df_stepped_u

['Mean_Z1-0', 'Mean_Z1-1', 'Mean_Z1-2', 'Mean_Z1-3', 'Mean_Z1-4', 'Mean_Z1-5', 'Mean_Z1-6', 'Mean_Z1-7', 'Mean_Z1-8', 'Mean_Z1-9', 'Mean_Z1-10', 'Std_Z1-0', 'Std_Z1-1', 'Std_Z1-2', 'Std_Z1-3', 'Std_Z1-4', 'Std_Z1-5', 'Std_Z1-6', 'Std_Z1-7', 'Std_Z1-8', 'Std_Z1-9', 'Std_Z1-10', 'Mean_Z2-0', 'Mean_Z2-1', 'Mean_Z2-2', 'Mean_Z2-3', 'Mean_Z2-4', 'Mean_Z2-5', 'Mean_Z2-6', 'Mean_Z2-7', 'Mean_Z2-8', 'Mean_Z2-9', 'Mean_Z2-10', 'Std_Z2-0', 'Std_Z2-1', 'Std_Z2-2', 'Std_Z2-3', 'Std_Z2-4', 'Std_Z2-5', 'Std_Z2-6', 'Std_Z2-7', 'Std_Z2-8', 'Std_Z2-9', 'Std_Z2-10', 'Mean_Z3-0', 'Mean_Z3-1', 'Mean_Z3-2', 'Mean_Z3-3', 'Mean_Z3-4', 'Mean_Z3-5', 'Mean_Z3-6', 'Mean_Z3-7', 'Mean_Z3-8', 'Mean_Z3-9', 'Mean_Z3-10', 'Std_Z3-0', 'Std_Z3-1', 'Std_Z3-2', 'Std_Z3-3', 'Std_Z3-4', 'Std_Z3-5', 'Std_Z3-6', 'Std_Z3-7', 'Std_Z3-8', 'Std_Z3-9', 'Std_Z3-10', 'Mean_Z4-0', 'Mean_Z4-1', 'Mean_Z4-2', 'Mean_Z4-3', 'Mean_Z4-4', 'Mean_Z4-5', 'Mean_Z4-6', 'Mean_Z4-7', 'Mean_Z4-8', 'Mean_Z4-9', 'Mean_Z4-10', 'Std_Z4-0', 'Std_Z4-1', 

,DATE,S.IndAll,Mean_Z1-0,Mean_Z1-1,Mean_Z1-2,Mean_Z1-3,Mean_Z1-4,Mean_Z1-5,Mean_Z1-6,Mean_Z1-7,...,Std_Z16-1,Std_Z16-2,Std_Z16-3,Std_Z16-4,Std_Z16-5,Std_Z16-6,Std_Z16-7,Std_Z16-8,Std_Z16-9,Std_Z16-10
0,1980-01-12,0.0,-1.696971,-1.515848,-1.803874,-1.880503,-1.60218,-1.724905,-1.884181,-1.374884,...,2.027211,2.227663,1.996661,2.803012,3.116646,2.578138,1.104857,0.798963,0.915706,1.077501
1,1980-01-13,0.0,-1.940168,-1.696971,-1.515848,-1.803874,-1.880503,-1.60218,-1.724905,-1.884181,...,1.542429,2.027211,2.227663,1.996661,2.803012,3.116646,2.578138,1.104857,0.798963,0.915706
2,1980-01-14,0.0,-2.275212,-1.940168,-1.696971,-1.515848,-1.803874,-1.880503,-1.60218,-1.724905,...,1.26081,1.542429,2.027211,2.227663,1.996661,2.803012,3.116646,2.578138,1.104857,0.798963
3,1980-01-15,0.0,-1.943522,-2.275212,-1.940168,-1.696971,-1.515848,-1.803874,-1.880503,-1.60218,...,1.476831,1.26081,1.542429,2.027211,2.227663,1.996661,2.803012,3.116646,2.578138,1.104857
4,1980-01-16,0.0,-2.001346,-1.943522,-2.275212,-1.940168,-1.696971,-1.515848,-1.803874,-1.880503,...,2.166364,1.476831,1.26081,1.542429,2.027211,2.227663,1.996661,2.803012,3.116646,2.578138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.5,-0.903472,-1.438548,-1.898231,-1.747452,-1.308732,-1.027514,-1.389268,-1.938775,...,1.207753,1.527343,0.81248,1.189631,1.313054,1.710854,1.742448,2.168615,2.24556,1.30885
15412,2022-03-24,1.0,-1.291725,-0.903472,-1.438548,-1.898231,-1.747452,-1.308732,-1.027514,-1.389268,...,1.249284,1.207753,1.527343,0.81248,1.189631,1.313054,1.710854,1.742448,2.168615,2.24556
15413,2022-03-25,1.0,-1.387864,-1.291725,-0.903472,-1.438548,-1.898231,-1.747452,-1.308732,-1.027514,...,1.331947,1.249284,1.207753,1.527343,0.81248,1.189631,1.313054,1.710854,1.742448,2.168615
15414,2022-03-26,1.0,-1.896455,-1.387864,-1.291725,-0.903472,-1.438548,-1.898231,-1.747452,-1.308732,...,1.94619,1.331947,1.249284,1.207753,1.527343,0.81248,1.189631,1.313054,1.710854,1.742448


In [18]:
max_steps = 11
labels_to_rewind = mean_std_atts_zones(16)
labels_stepped = attribute_stepped(labels_to_rewind, max_steps)
print(labels_stepped)

stepped_mean_zones = []
stepped_std_zones = []

for zone in range(1, 17):
  mean_press_10d = np.zeros((N_dates-max_steps, max_steps))
  std_press_10d = np.zeros((N_dates-max_steps, max_steps))

  mean_press = df_complete_v['Mean_Z{}'.format(zone)]
  std_press = df_complete_v['Std_Z{}'.format(zone)]

  for day in range(max_steps, N_dates):
    for steps in range(max_steps):
      mean_press_10d[day-max_steps, steps] = mean_press[day-steps]
      std_press_10d[day-max_steps, steps] = std_press[day-steps]
  print(mean_press_10d.shape)
  stepped_mean_zones.append(mean_press_10d)
  stepped_std_zones.append(std_press_10d)

final_array = np.hstack((target[max_steps:],
                         stepped_mean_zones[0], stepped_std_zones[0],
                         stepped_mean_zones[1], stepped_std_zones[1],
                         stepped_mean_zones[2], stepped_std_zones[2],
                         stepped_mean_zones[3], stepped_std_zones[3],
                         stepped_mean_zones[4], stepped_std_zones[4],
                         stepped_mean_zones[5], stepped_std_zones[5],
                         stepped_mean_zones[6], stepped_std_zones[6],
                         stepped_mean_zones[7], stepped_std_zones[7],
                         stepped_mean_zones[8], stepped_std_zones[8],
                         stepped_mean_zones[9], stepped_std_zones[9],
                         stepped_mean_zones[10], stepped_std_zones[10],
                         stepped_mean_zones[11], stepped_std_zones[11],
                         stepped_mean_zones[12], stepped_std_zones[12],
                         stepped_mean_zones[13], stepped_std_zones[13],
                         stepped_mean_zones[14], stepped_std_zones[14],
                         stepped_mean_zones[15], stepped_std_zones[15],
                         ))
df_stepped_v = pd.DataFrame(final_array , columns = target_att + labels_stepped)
df_stepped_v

['Mean_Z1-0', 'Mean_Z1-1', 'Mean_Z1-2', 'Mean_Z1-3', 'Mean_Z1-4', 'Mean_Z1-5', 'Mean_Z1-6', 'Mean_Z1-7', 'Mean_Z1-8', 'Mean_Z1-9', 'Mean_Z1-10', 'Std_Z1-0', 'Std_Z1-1', 'Std_Z1-2', 'Std_Z1-3', 'Std_Z1-4', 'Std_Z1-5', 'Std_Z1-6', 'Std_Z1-7', 'Std_Z1-8', 'Std_Z1-9', 'Std_Z1-10', 'Mean_Z2-0', 'Mean_Z2-1', 'Mean_Z2-2', 'Mean_Z2-3', 'Mean_Z2-4', 'Mean_Z2-5', 'Mean_Z2-6', 'Mean_Z2-7', 'Mean_Z2-8', 'Mean_Z2-9', 'Mean_Z2-10', 'Std_Z2-0', 'Std_Z2-1', 'Std_Z2-2', 'Std_Z2-3', 'Std_Z2-4', 'Std_Z2-5', 'Std_Z2-6', 'Std_Z2-7', 'Std_Z2-8', 'Std_Z2-9', 'Std_Z2-10', 'Mean_Z3-0', 'Mean_Z3-1', 'Mean_Z3-2', 'Mean_Z3-3', 'Mean_Z3-4', 'Mean_Z3-5', 'Mean_Z3-6', 'Mean_Z3-7', 'Mean_Z3-8', 'Mean_Z3-9', 'Mean_Z3-10', 'Std_Z3-0', 'Std_Z3-1', 'Std_Z3-2', 'Std_Z3-3', 'Std_Z3-4', 'Std_Z3-5', 'Std_Z3-6', 'Std_Z3-7', 'Std_Z3-8', 'Std_Z3-9', 'Std_Z3-10', 'Mean_Z4-0', 'Mean_Z4-1', 'Mean_Z4-2', 'Mean_Z4-3', 'Mean_Z4-4', 'Mean_Z4-5', 'Mean_Z4-6', 'Mean_Z4-7', 'Mean_Z4-8', 'Mean_Z4-9', 'Mean_Z4-10', 'Std_Z4-0', 'Std_Z4-1', 

,DATE,S.IndAll,Mean_Z1-0,Mean_Z1-1,Mean_Z1-2,Mean_Z1-3,Mean_Z1-4,Mean_Z1-5,Mean_Z1-6,Mean_Z1-7,...,Std_Z16-1,Std_Z16-2,Std_Z16-3,Std_Z16-4,Std_Z16-5,Std_Z16-6,Std_Z16-7,Std_Z16-8,Std_Z16-9,Std_Z16-10
0,1980-01-12,0.0,-1.267676,-1.342356,-1.257513,-1.621418,-1.061535,-1.194015,-0.427248,-1.089998,...,1.688812,1.016222,2.333677,1.418357,1.625481,1.090248,1.133739,1.104311,1.635449,1.789206
1,1980-01-13,0.0,-0.633654,-1.267676,-1.342356,-1.257513,-1.621418,-1.061535,-1.194015,-0.427248,...,1.652546,1.688812,1.016222,2.333677,1.418357,1.625481,1.090248,1.133739,1.104311,1.635449
2,1980-01-14,0.0,-0.802254,-0.633654,-1.267676,-1.342356,-1.257513,-1.621418,-1.061535,-1.194015,...,1.465097,1.652546,1.688812,1.016222,2.333677,1.418357,1.625481,1.090248,1.133739,1.104311
3,1980-01-15,0.0,-1.421811,-0.802254,-0.633654,-1.267676,-1.342356,-1.257513,-1.621418,-1.061535,...,1.710988,1.465097,1.652546,1.688812,1.016222,2.333677,1.418357,1.625481,1.090248,1.133739
4,1980-01-16,0.0,-0.479325,-1.421811,-0.802254,-0.633654,-1.267676,-1.342356,-1.257513,-1.621418,...,2.119534,1.710988,1.465097,1.652546,1.688812,1.016222,2.333677,1.418357,1.625481,1.090248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.5,2.131373,1.544517,1.27848,0.636975,0.150575,0.299077,-0.30915,-0.579079,...,1.34522,1.928095,2.395246,2.862885,3.038724,0.971094,1.775754,1.051605,2.245863,2.637944
15412,2022-03-24,1.0,2.224319,2.131373,1.544517,1.27848,0.636975,0.150575,0.299077,-0.30915,...,1.682846,1.34522,1.928095,2.395246,2.862885,3.038724,0.971094,1.775754,1.051605,2.245863
15413,2022-03-25,1.0,1.576384,2.224319,2.131373,1.544517,1.27848,0.636975,0.150575,0.299077,...,1.413358,1.682846,1.34522,1.928095,2.395246,2.862885,3.038724,0.971094,1.775754,1.051605
15414,2022-03-26,1.0,0.491059,1.576384,2.224319,2.131373,1.544517,1.27848,0.636975,0.150575,...,1.360739,1.413358,1.682846,1.34522,1.928095,2.395246,2.862885,3.038724,0.971094,1.775754


In [19]:
df_stepped_u.to_csv('./csv/16_zones/u_wind_10m_10D.csv', index=False)
df_stepped_v.to_csv('./csv/16_zones/v_wind_10m_10D.csv', index=False)